In [1]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#Workingfor a single collocation file. Testing for a whole day


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017jan/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date and time is: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' The INSAT file which was used is: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/jan2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017jan/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  2023-04-08 01:57:44.638689
 Collocated file date and time is:  02JAN2017_0415
 The INSAT file which was used is:  3RIMG_02JAN2017_0415_L1B_STD_V01R00.h5
 Intial Length of collocation file =  196
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017jan/day02/swr02JAN20170415_56825.csv
 Final Length of the collocation file is:  196
 The time taken for this file is:  208.47465586662292

 Collocated file date and time is:  02JAN2017_0545
 The INSAT file which was used is:  3RIMG_02JAN2017_0545_L1B_STD_V01R00.h5
 Intial Length of collocation file =  772
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017jan/day02/swr02JAN20170545_56826.csv
 Final Length of the collocation file is:  772
 The time taken for this file is:  807.4434952735901

 Collocated file date and time is:  02JAN2017_0715
 The INSAT file which was used is:  3RIMG_02JAN2017_07

In [2]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#SWIR retrieval for one whole day.


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017feb/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date-time: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' INSAT file which was used: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/feb2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017feb/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  08-Apr-2023 09:57:58.623053
 Collocated file date-time:  02FEB2017_0345
 INSAT file which was used:  3RIMG_02FEB2017_0345_L1B_STD_V01R00.h5
 Intial Length of collocation file =  20
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017feb/day02/swr02FEB20170345_57276.csv
 Final Length of the collocation file is:  20
 The time taken for this file is:  29.37462878227234

 Collocated file date-time:  02FEB2017_0445
 INSAT file which was used:  3RIMG_02FEB2017_0445_L1B_STD_V01R00.h5
 Intial Length of collocation file =  52
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017feb/day02/swr02FEB20170445_57277.csv
 Final Length of the collocation file is:  52
 The time taken for this file is:  62.25506830215454

 Collocated file date-time:  02FEB2017_0515
 INSAT file which was used:  3RIMG_02FEB2017_0515_L1B_STD_V01R00.h5
 Intial Length of colloca

In [3]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#SWIR retrieval for one whole day.


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017mar/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date-time: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' INSAT file which was used: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/mar2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017mar/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  08-Apr-2023 11:46:18.628355
 Collocated file date-time:  02MAR2017_0515
 INSAT file which was used:  3RIMG_02MAR2017_0515_L1B_STD_V01R00.h5
 Intial Length of collocation file =  258
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017mar/day02/swr02MAR20170515_57685.csv
 Final Length of the collocation file is:  258
 The time taken for this file is:  288.9047315120697

 Collocated file date-time:  02MAR2017_0545
 INSAT file which was used:  3RIMG_02MAR2017_0545_L1B_STD_V01R00.h5
 Intial Length of collocation file =  531
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017mar/day02/swr02MAR20170545_57685.csv
 Final Length of the collocation file is:  531
 The time taken for this file is:  593.0537285804749

 Collocated file date-time:  02MAR2017_0715
 INSAT file which was used:  3RIMG_02MAR2017_0715_L1B_STD_V01R00.h5
 Intial Length of col

In [4]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#SWIR retrieval for one whole day.


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017apr/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date-time: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' INSAT file which was used: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/apr2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017apr/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  08-Apr-2023 13:47:36.164062
 Collocated file date-time:  02APR2017_0315
 INSAT file which was used:  3RIMG_02APR2017_0315_L1B_STD_V01R00.h5
 Intial Length of collocation file =  5
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017apr/day02/swr02APR20170315_58135.csv
 Final Length of the collocation file is:  5
 The time taken for this file is:  14.672266006469727

 Collocated file date-time:  02APR2017_0445
 INSAT file which was used:  3RIMG_02APR2017_0445_L1B_STD_V01R00.h5
 Intial Length of collocation file =  385
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017apr/day02/swr02APR20170445_58136.csv
 Final Length of the collocation file is:  385
 The time taken for this file is:  421.07068610191345

 Collocated file date-time:  02APR2017_0515
 INSAT file which was used:  3RIMG_02APR2017_0515_L1B_STD_V01R00.h5
 Intial Length of collo

In [5]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#SWIR retrieval for one whole day.


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017may/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date-time: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' INSAT file which was used: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/may2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017may/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  08-Apr-2023 17:13:03.406025
 Collocated file date-time:  02MAY2017_0345
 INSAT file which was used:  3RIMG_02MAY2017_0345_L1B_STD_V01R00.h5
 Intial Length of collocation file =  11
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017may/day02/swr02MAY20170345_58572.csv
 Final Length of the collocation file is:  11
 The time taken for this file is:  20.258487462997437

 Collocated file date-time:  02MAY2017_0445
 INSAT file which was used:  3RIMG_02MAY2017_0445_L1B_STD_V01R00.h5
 Intial Length of collocation file =  174
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017may/day02/swr02MAY20170445_58573.csv
 Final Length of the collocation file is:  174
 The time taken for this file is:  185.63919138908386

 Collocated file date-time:  02MAY2017_0515
 INSAT file which was used:  3RIMG_02MAY2017_0515_L1B_STD_V01R00.h5
 Intial Length of col

In [6]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#SWIR retrieval for one whole day.


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017jun/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date-time: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' INSAT file which was used: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/jun2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017jun/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  08-Apr-2023 21:31:27.490744
 Collocated file date-time:  02JUN2017_0415
 INSAT file which was used:  3RIMG_02JUN2017_0415_L1B_STD_V01R00.h5
 Intial Length of collocation file =  214
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017jun/day02/swr02JUN20170415_59024.csv
 Final Length of the collocation file is:  214
 The time taken for this file is:  267.8192014694214

 Collocated file date-time:  02JUN2017_0445
 INSAT file which was used:  3RIMG_02JUN2017_0445_L1B_STD_V01R00.h5
 Intial Length of collocation file =  13
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017jun/day02/swr02JUN20170445_59024.csv
 Final Length of the collocation file is:  13
 The time taken for this file is:  24.756749629974365

 Collocated file date-time:  02JUN2017_0545
 INSAT file which was used:  3RIMG_02JUN2017_0545_L1B_STD_V01R00.h5
 Intial Length of coll

In [7]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#SWIR retrieval for one whole day.


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017nov/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date-time: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' INSAT file which was used: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/nov2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017nov/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  08-Apr-2023 23:24:06.648295
 Collocated file date-time:  02NOV2017_0415
 INSAT file which was used:  3RIMG_02NOV2017_0415_L1B_STD_V01R00.h5
 Intial Length of collocation file =  213
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017nov/day02/swr02NOV20170415_61252.csv
 Final Length of the collocation file is:  213
 The time taken for this file is:  241.72305583953857

 Collocated file date-time:  02NOV2017_0615
 INSAT file which was used:  3RIMG_02NOV2017_0615_L1B_STD_V01R00.h5
 Intial Length of collocation file =  95
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017nov/day02/swr02NOV20170615_61253.csv
 Final Length of the collocation file is:  95
 The time taken for this file is:  113.0134334564209

 Collocated file date-time:  02NOV2017_0715
 INSAT file which was used:  3RIMG_02NOV2017_0715_L1B_STD_V01R00.h5
 Intial Length of coll

In [8]:
import os
import pandas as pd
import numpy as np
import h5py
import time 
import pytz
import datetime
from pytz import timezone 
#SWIR retrieval for one whole day.


filesavingadress = r'/data/debasish/transferfiles/2017swirincluded/2017dec/day02'
def swiraddersinglefile(collocationfileadress, insatfolderadress):
    starttic = time.time()
    assert os.path.exists(collocationfileadress), "Collocation file does not exist"
    assert os.path.exists(insatfolderadress), "Insat folder does not exist"

    #Get full name of the col file from the adress
    colfilenamewithextension = collocationfileadress.split('/')[-1]

    insatfolderadress = insatfolderadress
    os.listdir(insatfolderadress)
    #Sort the files in ascending order
    insatfilescollection = sorted(os.listdir(insatfolderadress))

    #Extract which INSAT file is needed for the task

    colfilenoswiradress = collocationfileadress
    dateandtime = colfilenoswiradress.split('/')[-1].split('_')[0]
    #Delete col from dateandtime string
    dateandtime = dateandtime[3:] #To get 02JAN20170415
    dateandtime = dateandtime[0:9]+'_'+dateandtime[9:] #To get 02JAN2017_0445
    currentfiledateandtime = dateandtime
    print(" Collocated file date-time: ",currentfiledateandtime)
    matchedinsatadress = None #Assume that you won't find the file
    for i in range(len(insatfilescollection)):
        if currentfiledateandtime in insatfilescollection[i]:
            
            print(' INSAT file which was used: ',insatfilescollection[i])
            matchedinsatadress = insatfolderadress+'/'+insatfilescollection[i]
            break
    try:
        insatfile = h5py.File(matchedinsatadress,'r')
    except:
        matchedinsatadress = None
        print(' The INSAT file for col_file {} is not present in the list, check the date of INSAT folder'.format(currentfiledateandtime))
        return None
    
    #Prepare the INAST file for the task
    vislatitude = np.ma.masked_equal(insatfile["Latitude_VIS"],327670)/1000
    vislongitude = np.ma.masked_equal(insatfile["Longitude_VIS"],327670)/1000
    swir_img_arr = np.array(insatfile["IMG_SWIR"][0,:,:])
    swir_img_arr_fill = insatfile["IMG_SWIR"].attrs["_FillValue"][0]
    nan_mask = (swir_img_arr == swir_img_arr_fill)
    swir_rad_lut = np.array(insatfile["IMG_SWIR_RADIANCE"])
    
    def count2rad(count):
        return swir_rad_lut[count]
    swir_rad_arr = count2rad(swir_img_arr)
    swir_rad_arr[nan_mask] = np.nan

    #Extract SWIR radiance values for the collocation points
    swirradlist = []
    df = pd.read_csv(colfilenoswiradress)
    print(" Intial Length of collocation file = ",len(df))
    #Separate the latitude and longitude from the insatcorvis column and put it in two new columns
    df['insatcorvislat'] = df['insatcorvis'].apply(lambda x: x.split(',')[0])
    df['insatcorvislat'] = df['insatcorvislat'].apply(lambda x: x.split('(')[1]).astype(float)

    df['insatcorvislon'] = df['insatcorvis'].apply(lambda x: x.split(',')[1])
    df['insatcorvislon'] = df['insatcorvislon'].apply(lambda x: x.split(')')[0]).astype(float)

    for i in range(len(df)):
        lat = df['insatcorvislat'][i]
        lon = df['insatcorvislon'][i]
        #Find the index of the lat and lon in the insat file
        indices = np.argwhere((vislatitude == lat) & (vislongitude == lon))
        x_index = indices[0][0]
        y_index = indices[0][1]
        swirradlist.append(swir_rad_arr[x_index][y_index])
        print(i, end='\r')
    
    #Add the swir radiance values to the collocation file
    df['swirrad'] = swirradlist
    df['cloudyornot'] = df['thickness'].apply(lambda x: 1 if x>0 else 0)
    df['date'] = currentfiledateandtime.split('_')[0]
    df['time'] = currentfiledateandtime.split('_')[1]
    df.dropna(subset=['bttir1', 'bttir2', 'albedo', 'swirrad', 'thickness', 'cloudyornot', 'solarelevation'])

    #Save the collocation file with the swir radiance values
    colfilenamewithextension = colfilenamewithextension.replace('col','swr')
    df.to_csv(filesavingadress+'/'+colfilenamewithextension)
    print(' The collocation file with the swir radiance values is saved at: ',filesavingadress+'/'+colfilenamewithextension)
    print(' Final Length of the collocation file is: ',len(df))
    endtoc = time.time()
    print(' The time taken for this file is: ',endtoc-starttic)
    print()

#collocationfileadress = r'/data/debasish/collocations/2017/2017jan/day01/col01JAN20170345_56810.csv'

rootstart = time.time()
print("Program start (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
#swiraddersinglefile(collocationfileadress, insatfolderadress)

insatfolderadress = r'/data/debasish/insatdata/l1b/2017/dec2017/day02'
onedaycollocationfolder = r'/data/debasish/collocations/2017/2017dec/day02'
#Sort all the collocation files in the folder
onedaycollocationfoldercollection = sorted(os.listdir(onedaycollocationfolder))

#Apply the function to all the collocation files in the folder
for i in range(len(onedaycollocationfoldercollection)):
    collocationfileadress = onedaycollocationfolder+'/'+onedaycollocationfoldercollection[i]
    swiraddersinglefile(collocationfileadress, insatfolderadress)

rootend = time.time()
hours_taken = int((rootend-rootstart)/3600)
minutes_taken = int(((rootend-rootstart)-hours_taken*3600)/60)
seconds_taken = (rootend-rootstart)-hours_taken*3600-minutes_taken*60

print("Program end (IST) = ",datetime.datetime.now(timezone("Asia/Kolkata")).strftime('%d-%b-%Y %H:%M:%S.%f'))
print('The total time taken for the task is: ',hours_taken,' h',minutes_taken,' min ',seconds_taken,' sec')

Program start (IST) =  09-Apr-2023 01:17:50.766186
 Collocated file date-time:  02DEC2017_0415
 INSAT file which was used:  3RIMG_02DEC2017_0415_L1B_STD_V01R00.h5
 Intial Length of collocation file =  157
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017dec/day02/swr02DEC20170415_61689.csv
 Final Length of the collocation file is:  157
 The time taken for this file is:  188.1766722202301

 Collocated file date-time:  02DEC2017_0445
 INSAT file which was used:  3RIMG_02DEC2017_0445_L1B_STD_V01R00.h5
 Intial Length of collocation file =  82
 The collocation file with the swir radiance values is saved at:  /data/debasish/transferfiles/2017swirincluded/2017dec/day02/swr02DEC20170445_61689.csv
 Final Length of the collocation file is:  82
 The time taken for this file is:  103.54571986198425

 Collocated file date-time:  02DEC2017_0545
 INSAT file which was used:  3RIMG_02DEC2017_0545_L1B_STD_V01R00.h5
 Intial Length of coll